In [142]:
import pandas as pd

In [143]:
gca_df = pd.read_csv('data/gca_player_stats.csv')
defense_df = pd.read_csv('data/defense_player_stats.csv')
misc_df = pd.read_csv('data/misc_player_stats.csv')
passing_df = pd.read_csv('data/passing_player_stats.csv')
possession_df = pd.read_csv('data/possession_player_stats.csv')
shooting_df = pd.read_csv('data/shooting_player_stats.csv')


In [144]:
all_dfs = [gca_df, defense_df, misc_df, passing_df, possession_df, shooting_df]

for i in range(len(all_dfs)):
    all_dfs[i].columns = all_dfs[i].iloc[0]
    all_dfs[i] = all_dfs[i].iloc[1:].reset_index(drop=True)

In [145]:
# Правильный способ удаления колонок с NaN названиями
for i in range(len(all_dfs)):
    all_dfs[i] = all_dfs[i].loc[:, all_dfs[i].columns.notna()]

In [146]:
# Правильный способ обработки всех DataFrame'ов
for i in range(len(all_dfs)):
    # Удаляем строку с индексом 0 (если она существует)
    if 0 in all_dfs[i].index:
        all_dfs[i] = all_dfs[i].drop(index=0)
    
    # Удаляем колонку 'Rk' если она существует
    if 'Rk' in all_dfs[i].columns:
        all_dfs[i] = all_dfs[i].drop(columns=['Rk'])
    
    # Сбрасываем индекс
    all_dfs[i] = all_dfs[i].reset_index(drop=True)

In [147]:
possession_df.head()

,Rk,Player,Nation,Pos,Squad,Age,Born,90s,Touches,Def Pen,...,PrgC,1/3,CPA,Mis,Dis,Rec,PrgR,Matches,NaN,NaN
0,Rk,Player,Nation,Pos,Squad,Age,Born,90s,Touches,Def Pen,...,PrgC,1/3,CPA,Mis,Dis,Rec,PrgR,Matches,NaN,NaN
1,1,Max Aarons,eng ENG,DF,Bournemouth,24,2000,1.0,73,2,...,1,0,0,1,0,40,3,Matches,Premier League,possession
2,2,Joshua Acheampong,eng ENG,DF,Chelsea,18,2006,1.9,137,7,...,0,2,0,2,0,111,0,Matches,Premier League,possession
3,3,Tyler Adams,us USA,MF,Bournemouth,25,1999,21.8,1337,62,...,14,22,1,28,12,757,10,Matches,Premier League,possession
4,4,Tosin Adarabioyo,eng ENG,DF,Chelsea,26,1997,15.7,1330,175,...,5,2,0,4,0,1000,1,Matches,Premier League,possession


In [148]:
def clean(df):
       df = df.drop_duplicates(subset=['Squad', 'Player'])
       return df.reset_index(drop=True)

gca_df, defense_df, misc_df, passing_df, possession_df, shooting_df = \
       map(clean, [gca_df, defense_df, misc_df, passing_df, possession_df, shooting_df])
       


In [149]:

for i, df_name in enumerate(['shooting_df', 'passing_df', 'gca_df', 'defense_df', 'possession_df', 'misc_df']):
    df = [shooting_df, passing_df, gca_df, defense_df, possession_df, misc_df][i]
    
    if 'Player' in df.columns:
        header_rows = df[df['Player'] == 'Player'].index
        if len(header_rows) > 0:
            df = df.drop(header_rows)
            df = df.reset_index(drop=True)
        

In [150]:
from functools import reduce

def clean_dataframe(df):
    df_clean = df.copy()
    
    df_clean = df_clean.loc[:, df_clean.columns.notna()]
    
    df_clean = df_clean.loc[:, ~df_clean.columns.duplicated()]
    
    if 'Player' in df_clean.columns:
        df_clean = df_clean[df_clean['Player'] != 'Player']
    
    if 'Player' in df_clean.columns and 'Squad' in df_clean.columns:
        df_clean = df_clean.dropna(subset=['Player', 'Squad'])
        df_clean = df_clean[(df_clean['Player'] != '') & (df_clean['Squad'] != '')]
    
    if 'Player' in df_clean.columns and 'Squad' in df_clean.columns:
        df_clean = df_clean.drop_duplicates(subset=['Player', 'Squad'])
    
    df_clean = df_clean.reset_index(drop=True)
    
    return df_clean

dfs_clean = []
df_names = ['shooting_df', 'passing_df', 'gca_df', 'defense_df', 'possession_df', 'misc_df']
original_dfs = [shooting_df, passing_df, gca_df, defense_df, possession_df, misc_df]

for i, (df_name, df) in enumerate(zip(df_names, original_dfs)):
    df_cleaned = clean_dataframe(df)
    dfs_clean.append(df_cleaned)


In [151]:
common_cols = ['Rk', 'Nation', 'Pos', 'Age', 'Born', '90s', 'Matches']

for i in range(1, len(dfs_clean)):
    cols_to_keep = ['Player', 'Squad'] + [col for col in dfs_clean[i].columns 
                                         if col not in common_cols and col not in ['Player', 'Squad']]
    dfs_clean[i] = dfs_clean[i][cols_to_keep]

merged_df = reduce(lambda left, right: left.merge(right, on=['Player', 'Squad'], how='inner'), dfs_clean)
merged_df.head()


,Rk,Player,Nation,Pos,Squad,Age,Born,90s,Gls,Sh_x,...,Crs,Int_y,TklW_y,PKwon,PKcon,OG,Recov,Won,Lost_y,Won%
0,1,Max Aarons,eng ENG,DF,Bournemouth,24,2000,1.0,0,0,...,2,1,2,0,0,0,7,0,0,NaN
1,2,Joshua Acheampong,eng ENG,DF,Chelsea,18,2006,1.9,0,2,...,0,1,1,0,0,0,7,1,6,14.3
2,3,Tyler Adams,us USA,MF,Bournemouth,25,1999,21.8,0,9,...,3,32,50,0,1,0,114,31,18,63.3
3,4,Tosin Adarabioyo,eng ENG,DF,Chelsea,26,1997,15.7,1,13,...,0,11,13,0,0,0,41,42,28,60.0
4,5,Simon Adingra,ci CIV,"FW,MF",Brighton,22,2002,12.2,2,33,...,41,8,14,0,0,0,47,7,4,63.6


In [152]:
merged_df = merged_df.drop(columns = 'Rk')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2854 entries, 0 to 2853
Columns: 102 entries, Player to Won%
dtypes: object(102)
memory usage: 2.2+ MB


In [154]:
merged_df = merged_df.drop(columns = [col for col in merged_df.columns if '_x' in col or '_y' in col])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2854 entries, 0 to 2853
Data columns (total 78 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Player    2854 non-null   object
 1   Nation    2847 non-null   object
 2   Pos       2854 non-null   object
 3   Squad     2854 non-null   object
 4   Age       2846 non-null   object
 5   Born      2846 non-null   object
 6   90s       2854 non-null   object
 7   Gls       2854 non-null   object
 8   SoT       2854 non-null   object
 9   SoT%      2335 non-null   object
 10  Sh/90     2854 non-null   object
 11  SoT/90    2854 non-null   object
 12  G/Sh      2335 non-null   object
 13  G/SoT     1969 non-null   object
 14  Dist      2335 non-null   object
 15  FK        2854 non-null   object
 16  PK        2854 non-null   object
 17  PKatt     2854 non-null   object
 18  xG        2854 non-null   object
 19  npxG      2854 non-null   object
 20  npxG/Sh   2335 non-null   object
 21  G-xG      2854